**3. COLLECT THE DOUBLES COMPETITOR RANKINGS DATA FROM THE API ENDPOINTS**



In [47]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json"

headers = {
    "accept": "application/json",
    "x-api-key": "2gS1R2oRxMyHyBmFK4cJRK7N3HZIJRMXfDiVYRdv"
}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2025-05-02T19:05:11+00:00","rankings":[{"type_id":2,"name":"ATP","year":2025,"week":18,"gender":"men","competitor_rankings":[{"rank":1,"movement":0,"points":9350,"competitions_played":22,"competitor":{"id":"sr:competitor:49363","name":"Pavic, Mate","country":"Croatia","country_code":"HRV","abbreviation":"PAV"}},{"rank":1,"movement":0,"points":9350,"competitions_played":22,"competitor":{"id":"sr:competitor:51836","name":"Arevalo-Gonzalez, Marcelo","country":"El Salvador","country_code":"SLV","abbreviation":"ARE"}},{"rank":3,"movement":0,"points":7590,"competitions_played":25,"competitor":{"id":"sr:competitor:14898","name":"Heliovaara, Harri","country":"Finland","country_code":"FIN","abbreviation":"HEL"}},{"rank":4,"movement":0,"points":7590,"competitions_played":26,"competitor":{"id":"sr:competitor:637970","name":"Patten, Henry","country":"Great Britain","country_code":"GBR","abbreviation":"PAT"}},{"rank":5,"movement":0,"points":6460,"competitions_played":20,"competitor

In [ ]:
import json
data = json.loads(response.text)
rankings = data['rankings'][0]['competitor_rankings']

In [49]:
rankings_data = []
competitor_data = []

for ranking in rankings:
    rankings_data.append({
        'rank': ranking.get('rank'),
        'movement': ranking.get('movement'),
        'points': ranking.get('points'),
        'competitions_played': ranking.get('competitions_played'),
        'competitor_id': ranking['competitor'].get('id')
    })

for ranking in rankings:
    competitor = ranking['competitor']
    competitor_data.append({
        'competitor_id': competitor.get('id'),
        'name': competitor.get('name'),
        'country': competitor.get('country'),
        'country_code': competitor.get('country_code'),  
        'abbreviation': competitor.get('abbreviation')
    })

In [ ]:
import pandas as pd
rankings_df = pd.DataFrame(rankings_data)
competitor_df = pd.DataFrame(competitor_data)

In [78]:
import mysql.connector

conn_mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Joker@212814"
)
cursor_mysql = conn_mysql.cursor()
print("MySQL connection established!")

MySQL connection established!


In [86]:
cursor_mysql.execute("CREATE DATABASE IF NOT EXISTS tennis_db;")
print("MySQL database 'tennis_db' created successfully!")

MySQL database 'tennis_db' created successfully!


In [119]:
cursor_mysql.execute("USE tennis_db;")

cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS competitor (
        competitor_id VARCHAR(50) PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        country VARCHAR(100) NOT NULL,
        country_code CHAR(3) ,
        abbreviation VARCHAR(10) NOT NULL
    );
""")

cursor_mysql.execute("""
    CREATE TABLE IF NOT EXISTS rankings (
        rank_id INT AUTO_INCREMENT PRIMARY KEY,
        `rank` INT NOT NULL,
        movement INT NOT NULL,
        points INT NOT NULL,
        competitions_played INT NOT NULL,
        competitor_id VARCHAR(50),
        FOREIGN KEY (competitor_id) REFERENCES competitor(competitor_id)
    );
""")

conn_mysql.commit()
print("Tables 'rankings' and 'competitor' created successfully in MySQL!")


Tables 'rankings' and 'competitor' created successfully in MySQL!


In [120]:
for index, row in competitor_df.iterrows():
    cursor_mysql.execute("""
        INSERT INTO competitor (competitor_id, name, country, country_code, abbreviation)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            name = VALUES(name), 
            country = VALUES(country), 
            country_code = VALUES(country_code), 
            abbreviation = VALUES(abbreviation)
    """, (row['competitor_id'], row['name'], row['country'], row['country_code'], row['abbreviation']))

for index, row in rankings_df.iterrows():
    cursor_mysql.execute("""
        INSERT INTO rankings (`rank`, movement, points, competitions_played, competitor_id)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['rank'], row['movement'], row['points'], row['competitions_played'], row['competitor_id']))
    
conn_mysql.commit()
print("Data inserted successfully!")


Data inserted successfully!


1.Get all competitors with their rank and points.

In [151]:
from tabulate import tabulate

query1 = """
SELECT c.competitor_id , c.name , r.rank , r.points
FROM competitor c
JOIN rankings r ON c.competitor_id = r.competitor_id;
"""
cursor_mysql.execute(query1)
result = cursor_mysql.fetchall()

headers = ["Competitor_id", "Name", "Rank", "Points"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+-----------------------+--------------------------------------+--------+----------+
| Competitor_id         | Name                                 |   Rank |   Points |
+=======================+======================================+========+==========+
| sr:competitor:49363   | Pavic, Mate                          |      1 |     9350 |
+-----------------------+--------------------------------------+--------+----------+
| sr:competitor:51836   | Arevalo-Gonzalez, Marcelo            |      1 |     9350 |
+-----------------------+--------------------------------------+--------+----------+
| sr:competitor:14898   | Heliovaara, Harri                    |      3 |     7590 |
+-----------------------+--------------------------------------+--------+----------+
| sr:competitor:637970  | Patten, Henry                        |      4 |     7590 |
+-----------------------+--------------------------------------+--------+----------+
| sr:competitor:87690   | Thompson, Jordan                     | 

2.Find competitors ranked in the top 5

In [150]:
from tabulate import tabulate

query2 = """
SELECT c.competitor_id , c.name , r.rank
FROM competitor c
JOIN rankings r ON c.competitor_id = r.competitor_id
WHERE r.rank <= 5;
"""
cursor_mysql.execute(query2)
result = cursor_mysql.fetchall()

headers = ["Competitor_id", "Name", "Rank", "Points"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+----------------------+---------------------------+--------+
| Competitor_id        | Name                      |   Rank |
+======================+===========================+========+
| sr:competitor:49363  | Pavic, Mate               |      1 |
+----------------------+---------------------------+--------+
| sr:competitor:51836  | Arevalo-Gonzalez, Marcelo |      1 |
+----------------------+---------------------------+--------+
| sr:competitor:14898  | Heliovaara, Harri         |      3 |
+----------------------+---------------------------+--------+
| sr:competitor:637970 | Patten, Henry             |      4 |
+----------------------+---------------------------+--------+
| sr:competitor:87690  | Thompson, Jordan          |      5 |
+----------------------+---------------------------+--------+


3.List competitors with no rank movement (stable rank)

In [153]:
from tabulate import tabulate

query3 = """
SELECT c.competitor_id , c.name , r.rank , r.movement
FROM competitor c
JOIN rankings r ON c.competitor_id = r.competitor_id
WHERE movement = 0;
"""
cursor_mysql.execute(query3)
result = cursor_mysql.fetchall()

headers = ["Competitor_id", "Name", "Rank", "Movement"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+----------------------+---------------------------+--------+------------+
| Competitor_id        | Name                      |   Rank |   Movement |
+======================+===========================+========+============+
| sr:competitor:49363  | Pavic, Mate               |      1 |          0 |
+----------------------+---------------------------+--------+------------+
| sr:competitor:51836  | Arevalo-Gonzalez, Marcelo |      1 |          0 |
+----------------------+---------------------------+--------+------------+
| sr:competitor:14898  | Heliovaara, Harri         |      3 |          0 |
+----------------------+---------------------------+--------+------------+
| sr:competitor:637970 | Patten, Henry             |      4 |          0 |
+----------------------+---------------------------+--------+------------+
| sr:competitor:87690  | Thompson, Jordan          |      5 |          0 |
+----------------------+---------------------------+--------+------------+
| sr:competitor:15568  | 

4.Get the total points of competitors from a specific country (e.g., Croatia)

In [155]:
from tabulate import tabulate

query4 = """
SELECT c.country , SUM(r.points) as total_points
FROM rankings r
JOIN competitor c ON c.competitor_id = r.competitor_id
WHERE c.country = 'Croatia'
"""
cursor_mysql.execute(query4)
result = cursor_mysql.fetchall()

headers = ["Country","Total Points"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+-----------+----------------+
| Country   |   Total Points |
+===========+================+
| Croatia   |          15794 |
+-----------+----------------+


5.Count the number of competitors per country

In [144]:
from tabulate import tabulate

query5 = """
SELECT country, count(competitor_id) as total_competitor
FROM competitor
GROUP BY country
"""
cursor_mysql.execute(query5)
result = cursor_mysql.fetchall()

headers = ["Country","Total Competitors"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+--------------------------+---------------------+
| Country                  |   Total Competitors |
+==========================+=====================+
| Ukraine                  |                   6 |
+--------------------------+---------------------+
| Turkiye                  |                   4 |
+--------------------------+---------------------+
| USA                      |                  44 |
+--------------------------+---------------------+
| Chile                    |                   4 |
+--------------------------+---------------------+
| Netherlands              |                  17 |
+--------------------------+---------------------+
| Venezuela                |                   4 |
+--------------------------+---------------------+
| Belgium                  |                   4 |
+--------------------------+---------------------+
| Germany                  |                  20 |
+--------------------------+---------------------+
| Italy                    |   

6.Find competitors with the highest points in the current week

In [158]:
from tabulate import tabulate

query6 = """
SELECT c.competitor_id, c.name ,r.points
FROM competitor c
JOIN rankings r ON c.competitor_id = r.competitor_id
WHERE r.points = (SELECT MAX(points) FROM rankings)
"""
cursor_mysql.execute(query6)
result = cursor_mysql.fetchall()

headers = ["Country","Total Competitors"]

print(tabulate(result, headers=headers, tablefmt="grid"))

+---------------------+---------------------------+---------------------+
|                     | Country                   |   Total Competitors |
+=====================+===========================+=====================+
| sr:competitor:49363 | Pavic, Mate               |                9350 |
+---------------------+---------------------------+---------------------+
| sr:competitor:51836 | Arevalo-Gonzalez, Marcelo |                9350 |
+---------------------+---------------------------+---------------------+
